### 1. Conceptos fundamentales

**Modelo causal de lenguaje:**  
Un modelo como Llama 3.2-1B predice el siguiente token dado un contexto anterior. Está formado por múltiples capas, cada una con un bloque de atención seguido de un MLP (una red lineal‑no‑lineal‑lineal).

**Capa MLP-8:**  
Corresponde a la novena capa del modelo (índice 8 porque se cuenta desde cero). El objetivo es capturar la salida de esta capa para cada token procesado.

**Token:**  
Es un número entero que representa una palabra, subpalabra o carácter en el vocabulario del modelo. Un texto se tokeniza en una secuencia de estos enteros.

**Token especial:**  
Incluyen `<bos>`, `<eos>`, `<pad>`, etc. Son usados para marcar inicios, finales o estructuras sintácticas. Se eliminan del análisis para evitar sesgos.

**Activación:**  
Vector numérico que representa cómo el modelo codifica la información para un token dado. La salida de la capa MLP-8 tiene dimensión 2048.

**Hook:**  
Función que intercepta la salida de una parte del modelo sin alterar su comportamiento. Se registra sobre `mlp` de la capa 8 para capturar su activación.

### 2. Preparación del entorno

El script realiza configuraciones esenciales:

- Establece la variable `HF_TOKEN` para autenticarse en Hugging Face.
- Define el modelo y dispositivo (`cuda`, `cpu` o `mps`).
- Ajusta el tipo de dato (`float16`, `bfloat16` o `float32`) según soporte del sistema.
- Carga el tokenizador y modelo desde Hugging Face.

```python
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype if DEVICE == "cuda" else torch.float32,
    device_map="auto" if DEVICE == "cuda" else None,
).eval()
```



### 3. Registro del hook

```python
mlp_out = None
def hook(_m, _i, o):
    global mlp_out
    mlp_out = o.detach()
model.model.layers[8].mlp.register_forward_hook(hook)
```

Este código asegura que, al pasar datos por el modelo, las activaciones de la capa MLP‑8 se guarden automáticamente en `mlp_out`.



### 4. Preprocesamiento de texto y segmentación

Se carga el corpus `pile-uncopyrighted` en modo streaming, lo que permite procesar ejemplos sin saturar la memoria.

#### Filtrado de calidad

```python
def good(t): t = t.strip(); return 20 < len(t) < 30_000 and t.upper() != t and not t.isnumeric()
```

Esta función descarta textos vacíos, completamente en mayúsculas o puramente numéricos.

#### División en bloques

```python
def chunker(text, did):
    ids = tok(text, add_special_tokens=False, truncation=True, max_length=TRUNC_TOKS).input_ids
    seq = [tok.bos_token_id] + ids + [tok.eos_token_id]
    for s in range(0, len(seq) - SEQ_LEN + 1, SEQ_LEN):
        yield seq[s:s+SEQ_LEN], did, s
```
Cada documento válido se divide en segmentos de longitud fija (4096 tokens), respetando los límites del modelo.


### 5. Paso por el modelo y filtrado

El modelo se evalúa en lotes (`BATCH_FWD = 16`), y se captura la activación `mlp_out`. Luego:

- Se filtran los tokens especiales.
- Se convierten las activaciones a `torch.uint16` tras proyectarlas desde `bfloat16`.
- Se construye un diccionario con metadatos (`doc_id`, `tok_pos`, `token_id`) y la activación comprimida.

```python
acts_k = acts[i][mask[i]].to(dtype).view(torch.uint16).cpu()
```

### 6. Construcción del buffer y cálculo parcial de RMS

Por cada token válido, se añade una entrada al buffer:

```python
{
  "doc_id": doc_buf[i],
  "tok_pos": pos_buf[i] + j,
  "token_id": int(tid),
  "activacion": bits.numpy()
}
```

Simultáneamente, se actualizan acumuladores para el cálculo de la raíz cuadrada media (RMS):

```python
sum_sq += acts_k.to(torch.float32).pow(2).sum()
n_tok  += len(acts_k)
```

### 7. Shardeo y subida a Hugging Face

Cuando el buffer alcanza `SHARD_SIZE = 50_000` ejemplos:

- Se crea un shard con `Dataset.from_list(...)`.
- Se sube al repositorio Hugging Face con `.push_to_hub(...)`.
- Si falla, se guarda localmente en `/workspace/fallback_...`.

```python
Dataset.from_list(push_buf, features=features).push_to_hub(REPO_ID, split=split, ...)
```

### 8. Procesamiento final y cómputo del RMS

Una vez que se han procesado todos los documentos:

- Se calcula el RMS global como:

```python
rms = math.sqrt(sum_sq / n_tok)
```

- Se guarda en un archivo `norm_stats.json`.
- Se sube al repositorio con `HfApi().upload_file(...)`.

Esto proporciona una métrica de escala útil para normalizar las activaciones más adelante.



### 9. Definición de un shard

Un **shard** es una porción independiente del dataset total, que contiene hasta `SHARD_SIZE = 50_000` ejemplos. Cada ejemplo representa un token válido y sus metadatos:

```json
{
  "doc_id": 123,
  "tok_pos": 2048,
  "token_id": 4871,
  "activacion": [52839, 39284, 15473, ..., 18924]  // vector codificado en uint16
}
```

La activación se almacena comprimida en `uint16`, reinterpretando los `bfloat16` originales sin perder información binaria, para ahorrar espacio.


### 10. Subida a Hugging Face

Cuando el buffer `push_buf` alcanza `SHARD_SIZE` filas, se ejecuta:

```python
Dataset.from_list(push_buf, features=features).push_to_hub(
    REPO_ID, split=split, private=True, max_shard_size="500MB"
)
```

Este código:

- Crea un `Dataset` estructurado con `Features` fijos.
- Lo sube al repositorio `naraca/activaciones-llama3-mlp8` en el split nombrado `train_XXXXX`.
- Usa shards privados de tamaño máximo 500MB para facilitar el acceso y evitar fragmentación.


### 11. Manejo de errores en la subida

Si ocurre una excepción durante la subida (por ejemplo, por red lenta o errores del Hub), se ejecuta el bloque alternativo:

```python
Dataset.from_list(push_buf, features=features).save_to_disk(f"/workspace/fallback_{split}")
```

Esto asegura que **ningún shard se pierda**, incluso si la conexión falla. Puedes posteriormente subir manualmente esos shards de respaldo.


### 12. Proceso final y restos

Después del bucle principal, es posible que queden datos no procesados en `mini`, `doc_buf` y `push_buf`. Se ejecuta el mismo procedimiento para esos datos restantes:

- Se procesan los ejemplos con el modelo.
- Se calcula su activación.
- Se filtran y empaquetan.
- Se suben como el shard final.

Esto garantiza que el dataset sea exhaustivo y que no se pierda información útil.


### 13. Cálculo del RMS global

Para normalizar las activaciones en futuros entrenamientos, se calcula la raíz cuadrada media (RMS):

```python
rms = math.sqrt(sum_sq / n_tok)
```

Donde:

- `sum_sq` es la suma total de cuadrados de las activaciones válidas.
- `n_tok` es el número total de tokens considerados.

Este valor se guarda en un archivo `.json` como:

```json
{
  "tokens": 3201451,
  "rms": 1.7294
}
```

Y se sube al mismo repositorio como `norm_stats.json`.


### 14. Diseño final del dataset

El dataset resultante tiene la siguiente estructura:

- **Split:** `train_00000`, `train_00001`, ..., uno por shard.
- **Campos por fila:**
  - `"doc_id"`: ID del documento original.
  - `"tok_pos"`: posición del token dentro del documento.
  - `"token_id"`: ID del token según el vocabulario del modelo.
  - `"activacion"`: lista de 2048 valores `uint16` representando el vector `bfloat16`.

Esto permite:

- Cargar secuencias alineadas con sus activaciones.
- Reconstruir texto si es necesario (usando `token_id`).
- Entrenar autoencoders o modelos supervisados.


### 15. Verificación de calidad

Puedes evaluar la calidad de un shard al cargarlo desde Hugging Face con:

```python
from datasets import load_dataset

ds = load_dataset("naraca/activaciones-llama3-mlp8", split="train_00000")
print(ds[0])
```

También puedes verificar la dispersión de activaciones, histograma de tokens o distribución de posiciones para asegurar una cobertura representativa del corpus.
